# Thesis Code Analysis - Plotting Figures and Such

## Importing Toolboxes

In [ ]:
# Import toolboxes
import matplotlib
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import pandas as pd
import datetime
import glob
import os

## Importing/Formatting Biomass Data

Credit to Dephine Mossman for the Derived Biomass dataset from the raw AZFP record onboard RU39. Without it, none of this would be possible!

In [ ]:
# list all csv files only, then concat
csv_files_1 = glob.glob('Derived Biomass Data 4_20/*.csv')
csv_files_2 = glob.glob('Derived Biomass Data 11_3/*.csv')
csv_files_3 = glob.glob('Derived Biomass Data 2_15/*.csv')

In [ ]:
concat_1 = pd.concat(map(pd.read_csv, csv_files_1), ignore_index=True)
concat_2 = pd.concat(map(pd.read_csv, csv_files_2), ignore_index=True)
concat_3 = pd.concat(map(pd.read_csv, csv_files_3), ignore_index=True)

In [ ]:
# Read in the Obs CSV for the dataset collecting in April 20th
apr_df = concat_1[concat_1['Frequency'] == 455]
# Do the same for the datasets from November 3rd and Febuary 15th
nov_df = concat_2[concat_2['Frequency'] == 455]
nov_df = concat_2.drop(concat_2[concat_2['Species'] == 'Larvacean'].index)
nov_df = concat_2.drop(concat_2[concat_2['Species'] == 'Salp'].index)
feb_df = concat_3[concat_3['Frequency'] == 455]
feb_df = concat_3.drop(concat_3[concat_3['Species'] == 'Salp'].index)

In [ ]:
# Create Log10 values
apr_df['Abundance_base10'] = np.log10(apr_df['Abundance'])
nov_df['Abundance_base10'] = np.log10(nov_df['Abundance'])
feb_df['Abundance_base10'] = np.log10(feb_df['Abundance'])
apr_df['Biomass_base10'] = np.log10(apr_df['Biomass'])
nov_df['Biomass_base10'] = np.log10(nov_df['Biomass'])
feb_df['Biomass_base10'] = np.log10(feb_df['Biomass'])

In [ ]:
# Convert "datetime" dataset to Datetime format
apr_df['timestamp'] = pd.to_datetime(apr_df['Date_M']+ ' ' + apr_df['Time_M'], format='%Y-%m-%d %H:%M:%S.%f', infer_datetime_format=True)
nov_df['timestamp'] = pd.to_datetime(nov_df['Date_M']+ ' ' + nov_df['Time_M'], format='%Y-%m-%d %H:%M:%S.%f' ,infer_datetime_format=True)
feb_df['timestamp'] = pd.to_datetime(feb_df['Date_M']+ ' ' + feb_df['Time_M'], format='%Y-%m-%d %H:%M:%S.%f' ,infer_datetime_format=True)

In [ ]:
# apr_df.to_csv(r"C:\Users\nixo1\Desktop\Thesis_Folder\apr_df.csv", index=False)
# nov_df.to_csv(r"C:\Users\nixo1\Desktop\Thesis_Folder\nov_df.csv", index=False)
# feb_df.to_csv(r"C:\Users\nixo1\Desktop\Thesis_Folder\feb_df.csv", index=False)

## Importing & Formatting the ERDDAP Datasets

In [ ]:
# Load in the URL for the Delayed Science data from tabledap 
url1  = 'https://slocum-data.marine.rutgers.edu/erddap/tabledap/ru39-20230420T1636-profile-sci-delayed.csv?time%2Clatitude%2Clongitude%2Cdepth%2Cbeta_700nm%2Ccdom%2Cchlorophyll_a%2Cconductivity%2Cdensity%2Cinstrument_ph%2CpH%2Cpressure%2Csalinity%2Ctemperature&time%3E=2023-04-20T16%3A36%3A28.28Z&time%3C=2023-05-12T18%3A10%3A56.60Z'
url2  = 'https://slocum-data.marine.rutgers.edu/erddap/tabledap/ru39-20231103T1413-profile-sci-delayed.csv?time%2Clatitude%2Clongitude%2Cdepth%2Cbeta_700nm%2Ccdom%2Cchlorophyll_a%2Cconductivity%2Cdensity%2Cinstrument_ph%2CpH%2Cpressure%2Csalinity%2Ctemperature&time%3E=2023-11-03T14%3A13%3A03.00Z&time%3C=2023-12-05T20%3A53%3A59.89Z'
url3  = 'https://slocum-data.marine.rutgers.edu/erddap/tabledap/ru39-20240215T1646-profile-sci-delayed.csv?time%2Clatitude%2Clongitude%2Cdepth%2Cbeta_700nm%2Ccdom%2Cchlorophyll_a%2Cconductivity%2Cdensity%2Cinstrument_ph%2CpH%2Cpressure%2Csalinity%2Ctemperature&time%3E=2024-02-15T16%3A46%3A42.00Z&time%3C=2024-03-14T18%3A15%3A07.82Z'

In [ ]:
DE_420 = pd.read_csv(url1, sep=',', skiprows=range(1, 2), parse_dates=['time'])
DE_1103 = pd.read_csv(url2, sep=',', skiprows=range(1, 2), parse_dates=['time'])
DE_215 = pd.read_csv(url3, sep=',', skiprows=range(1, 2), parse_dates=['time'])

In [ ]:
DE_420['timestamp'] = DE_420['time'].dt.tz_localize(None)
DE_1103['timestamp'] = DE_1103['time'].dt.tz_localize(None)
DE_215['timestamp'] = DE_215['time'].dt.tz_localize(None)

## Combining the time stamps!

In [ ]:
## Start by matching the time values under the name "timestamp"
DE_420 = DE_420.sort_values(['timestamp'])
DE_1103 = DE_1103.sort_values(['timestamp'])
DE_215 = DE_215.sort_values(['timestamp'])
apr_df = apr_df.sort_values(['timestamp'])
nov_df = nov_df.sort_values(['timestamp'])
feb_df = feb_df.sort_values(['timestamp'])

In [ ]:
###### This is the secret sauce that sorts the nearest timestamps between the two datasets
apr_mix = pd.merge_asof(apr_df, DE_420, on="timestamp", direction="nearest")
nov_mix = pd.merge_asof(nov_df, DE_1103, on="timestamp", direction="nearest")
feb_mix = pd.merge_asof(feb_df, DE_215, on="timestamp", direction="nearest")

In [ ]:
# apr_mix.to_csv(r"C:\Users\nixo1\Desktop\Thesis_Folder\apr_mix.csv")
# nov_mix.to_csv(r"C:\Users\nixo1\Desktop\Thesis_Folder\nov_mix.csv")
# feb_mix.to_csv(r"C:\Users\nixo1\Desktop\Thesis_Folder\feb_mix.csv")

## Obtaining Mixed Depth Values

From "Lori_Code.ipynb" I can read the resulting NetCFD files as CSV. The original source code for conversion of file formats comes from: https://gist.github.com/copernicusmarinegist/b57417225d0d4ea47c5d6200f9d8cac3

In [ ]:
# Replace 'local_storage_directory', 'netcdf_dir' and 'csv_dir'
# the directory path, the directory path to netcdf files and the directory path to csv files
local_storage_directory = r'/Users/nixo1/Desktop/Thesis_Folder/MLD/'
netcdf_dir = local_storage_directory + r'NetCDF/'
csv_dir = local_storage_directory + r'CSV/'

In [ ]:
# Set a loop to create a .csv file for each .nc file listed in `files_to_convert`
for filename in os.listdir(netcdf_dir):
    ds = xr.open_dataset(netcdf_dir + filename)
    df = ds.to_dataframe()
    df.to_csv(csv_dir + filename[:-3] + '.csv')
    print (filename + ' has been processed to .csv')

In [ ]:
MLD_420 = pd.read_csv(r"C:\Users\nixo1\Desktop\Thesis_Folder\MLD\CSV\ru39-20230420-mld.csv", sep=',', skiprows=range(1, 2), parse_dates=['time'])
MLD_1103 = pd.read_csv(r"C:\Users\nixo1\Desktop\Thesis_Folder\MLD\CSV\ru39-20231103-mld.csv", sep=',', skiprows=range(1, 2), parse_dates=['time'])

Because of the all 'NaN' values, the Winter data set had to be split into two seperate df which can then be spliced back together. This may or may not be neccesary consideing that the winter profile technically had no mixed layer at all.

In [ ]:
Slice1 = pd.read_csv(r"C:\Users\nixo1\Desktop\Thesis_Folder\MLD\CSV\ru39-20240215-slice1-mld.csv", sep=',', skiprows=range(1, 2), parse_dates=['time'])
Slice2 = pd.read_csv(r"C:\Users\nixo1\Desktop\Thesis_Folder\MLD\CSV\ru39-20240215-slice2-mld.csv", sep=',', skiprows=range(1, 2), parse_dates=['time'])
MLD_215 = pd.concat([Slice1,Slice2], ignore_index=False)

In [ ]:
MLD_420['timestamp'] = MLD_420['time'].dt.tz_localize(None)
MLD_1103['timestamp'] = MLD_1103['time'].dt.tz_localize(None)
MLD_215['timestamp'] = MLD_215['time'].dt.tz_localize(None)

MLD_420 = MLD_420.sort_values(['timestamp'])
MLD_1103 = MLD_1103.sort_values(['timestamp'])
MLD_215 = MLD_215.sort_values(['timestamp'])

In [ ]:
# Sort the nearest timestamps between the two datasets
apr_mix_mld = pd.merge_asof(apr_df, MLD_420, on="timestamp", direction="nearest")
apr_mix_mld.to_csv(r"C:\Users\nixo1\Desktop\Thesis_Folder\apr_mix_mld.csv")
nov_mix_mld = pd.merge_asof(nov_df, MLD_1103, on="timestamp", direction="nearest")
nov_mix_mld.to_csv(r"C:\Users\nixo1\Desktop\Thesis_Folder\nov_mix_mld.csv")
feb_mix_mld = pd.merge_asof(feb_df, MLD_215, on="timestamp", direction="nearest")
feb_mix_mld.to_csv(r"C:\Users\nixo1\Desktop\Thesis_Folder\feb_mix_mld.csv")

## Organizing of Isobaths

In [ ]:
# Create seasons to help divide by isobath 
apr_mix['isobath'] = 'Spring 2023'
nov_mix['isobath'] = 'Autumn 2023'
feb_mix['isobath'] = 'Winter 2024'

#Depth related data can be presented when diviing up the values into four distinct isobaths
isobath_A1 = apr_mix.loc[(apr_mix['depth'] <= 20)]
isobath_A2 = apr_mix.loc[(apr_mix['depth'] >= 20) & (apr_mix['depth'] <= 40)]
isobath_A3 = apr_mix.loc[(apr_mix['depth'] >= 40) & (apr_mix['depth'] <= 60)]
isobath_A4 = apr_mix.loc[(apr_mix['depth'] >= 60)]

isobath_B1 = nov_mix.loc[(nov_mix['depth'] <= 20)]
isobath_B2 = nov_mix.loc[(nov_mix['depth'] >= 20) & (nov_mix['depth'] <= 40)]
isobath_B3 = nov_mix.loc[(nov_mix['depth'] >= 40) & (nov_mix['depth'] <= 60)]
isobath_B4 = nov_mix.loc[(nov_mix['depth'] >= 60)]

isobath_C1 = feb_mix.loc[(feb_mix['depth'] <= 20)]
isobath_C2 = feb_mix.loc[(feb_mix['depth'] >= 20) & (feb_mix['depth'] <= 40)]
isobath_C3 = feb_mix.loc[(feb_mix['depth'] >= 40) & (feb_mix['depth'] <= 60)]
isobath_C4 = feb_mix.loc[(feb_mix['depth'] >= 60)]

Reorganize data to seperate by isobath:

In [ ]:
Isobath_20 = pd.concat([isobath_A1, isobath_B1, isobath_C1], axis=0)
Isobath_20.replace([-np.inf], np.nan, inplace=True)
Isobath_20 = Isobath_20[Isobath_20['Biomass_base10'].notna()]
Isobath_20 = Isobath_20.reset_index(drop=True)
Isobath_20.to_csv(r"C:\Users\nixo1\Desktop\Thesis_Folder\Isobath_20.csv")

In [ ]:
Isobath_40 = pd.concat([isobath_A2, isobath_B2, isobath_C2], axis=0)
Isobath_40.replace([-np.inf], np.nan, inplace=True)
Isobath_40 = Isobath_40[Isobath_40['Biomass_base10'].notna()]
Isobath_40 = Isobath_40.reset_index(drop=True)
Isobath_40.to_csv(r"C:\Users\nixo1\Desktop\Thesis_Folder\Isobath_40.csv")

In [ ]:
Isobath_60 = pd.concat([isobath_A3, isobath_B3, isobath_C3], axis=0)
Isobath_60.replace([-np.inf], np.nan, inplace=True)
Isobath_60 = Isobath_60[Isobath_60['Biomass_base10'].notna()]
Isobath_60 = Isobath_60.reset_index(drop=True)
Isobath_60.to_csv(r"C:\Users\nixo1\Desktop\Thesis_Folder\Isobath_60.csv")

In [ ]:
Isobath_max = pd.concat([isobath_A4, isobath_B4, isobath_C4], axis=0)
Isobath_max.replace([-np.inf], np.nan, inplace=True)
Isobath_max = Isobath_max[Isobath_max['Biomass_base10'].notna()]
Isobath_max = Isobath_max.reset_index(drop=True)
Isobath_max.to_csv(r"C:\Users\nixo1\Desktop\Thesis_Folder\Isobath_max.csv")

## Violin Plots (Depth Bin Averages)

Out of the various types of plot, my advisors agreed that a violing plot is the most approraate way of showing the different isobaths by season. Statistical anaylsis can then be performed afterwards.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set()
fig, (ax1,ax2,ax3,ax4) = plt.subplots(1,4, figsize=(18, 5))
sns.violinplot(x="isobath", y="Abundance_base10", data=Isobath_20, cut=0, ax=ax1)
sns.despine(left=True)
sns.violinplot(x="isobath", y="Abundance_base10", data=Isobath_40, cut=0, ax=ax2)
sns.despine(left=True)
sns.violinplot(x="isobath", y="Abundance_base10", data=Isobath_60, cut=0, ax=ax3)
sns.despine(left=True)
sns.violinplot(x="isobath", y="Abundance_base10", data=Isobath_max, cut=0, ax=ax4)
sns.despine(left=True)
plt.savefig("violin1.png")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set()
fig, (ax1,ax2,ax3,ax4) = plt.subplots(1,4, figsize=(18, 5))
sns.violinplot(x="isobath", y="Biomass_base10", data=Isobath_20, cut=0, ax=ax1)
sns.despine(left=True)
sns.violinplot(x="isobath", y="Biomass_base10", data=Isobath_40, cut=0, ax=ax2)
sns.despine(left=True)
sns.violinplot(x="isobath", y="Biomass_base10", data=Isobath_60, cut=0, ax=ax3)
sns.despine(left=True)
sns.violinplot(x="isobath", y="Biomass_base10", data=Isobath_max, cut=0, ax=ax4)
sns.despine(left=True)
plt.savefig("violin2.png")

## Scatterplots

In [ ]:
fig,axs = plt.subplots(3,2, figsize=(15,12))

axs[0,0].scatter(apr_mix['temperature'],apr_mix['Abundance_base10'],10,marker ="o")
axs[0,0].scatter(nov_mix['temperature'],nov_mix['Abundance_base10'],10,marker ="s")
axs[0,0].scatter(feb_mix['temperature'],feb_mix['Abundance_base10'],10,marker ="^")
axs[0,0].set(xlabel='Temperature [°C]')

axs[0,1].scatter(apr_mix['salinity'],apr_mix['Abundance_base10'],10,marker ="o")
axs[0,1].scatter(nov_mix['salinity'],nov_mix['Abundance_base10'],10,marker ="s")
axs[0,1].scatter(feb_mix['salinity'],feb_mix['Abundance_base10'],10,marker ="^")
axs[0,1].set(xlabel='Salinity [psu]')

axs[1,0].scatter(apr_mix['density'],apr_mix['Abundance_base10'],10,marker ="o")
axs[1,0].scatter(nov_mix['density'],nov_mix['Abundance_base10'],10,marker ="s")
axs[1,0].scatter(feb_mix['density'],feb_mix['Abundance_base10'],10,marker ="^")
axs[1,0].set(xlabel='Density [kg m-1 s-1]')

axs[1,1].scatter(apr_mix['chlorophyll_a'],apr_mix['Abundance_base10'],10,marker ="o")
axs[1,1].scatter(nov_mix['chlorophyll_a'],nov_mix['Abundance_base10'],10,marker ="s")
axs[1,1].scatter(feb_mix['chlorophyll_a'],feb_mix['Abundance_base10'],10,marker ="^")
axs[1,1].set(xlabel='Chlorophyll [μg L-1]')

axs[2,0].scatter(apr_mix['pH'],apr_mix['Abundance_base10'],10,marker ="o")
axs[2,0].scatter(nov_mix['pH'],nov_mix['Abundance_base10'],10,marker ="s")
axs[2,0].scatter(feb_mix['pH'],feb_mix['Abundance_base10'],10,marker ="^")
axs[2,0].set(xlabel='pH')

axs[2,1].scatter(apr_mix['cdom'],apr_mix['Abundance_base10'],10,marker ="o")
axs[2,1].scatter(nov_mix['cdom'],nov_mix['Abundance_base10'],10,marker ="s")
axs[2,1].scatter(feb_mix['cdom'],feb_mix['Abundance_base10'],10,marker ="^")
axs[2,1].set(xlabel='CDOM')

for ax in axs.flat:
    ax.set(ylabel='Log10 of Abundance [individuals per m$^{{-3}}$]')
    ax.legend(["Spring 2023" , "Fall 2023", "Winter 2024"])
plt.tight_layout()
plt.savefig("sixplots.png")

### April 20th to May 12th

In [ ]:
fig,axs = plt.subplots(3,2, figsize=(15,12))

axs_1=axs[0,0].scatter(apr_mix['temperature'],apr_mix['Abundance_base10'],10,c = apr_df['timestamp'],cmap='cool')
# axs[0,0].set_title('Log10 of Abundance [individuals per m$^{{-3}}$]')
axs[0,0].set(xlabel='Temperature [°C]')

axs_2=axs[0,1].scatter(apr_mix['salinity'],apr_mix['Abundance_base10'],10,c = apr_df['timestamp'],cmap='cool')
# axs[0,1].set_title('Log10 of Abundance [individuals per m$^{{-3}}$]')
axs[0,1].set(xlabel='Salinity [psu]')

axs_3=axs[1,0].scatter(apr_mix['density'],apr_mix['Abundance_base10'],10,c = apr_df['timestamp'],cmap='cool')
# axs[1,0].set_title('Log10 of Abundance [individuals per m$^{{-3}}$]')
axs[1,0].set(xlabel='Density [kg m-1 s-1]')

axs_4=axs[1,1].scatter(apr_mix['chlorophyll_a'],apr_mix['Abundance_base10'],10,c = apr_df['timestamp'],cmap='cool')
# axs[1,1].set_title('Log10 of Abundance [individuals per m$^{{-3}}$]')
axs[1,1].set(xlabel='Chlorophyll [μg L-1]')

axs_5=axs[2,0].scatter(apr_mix['pH'],apr_mix['Abundance_base10'],10,c = apr_df['timestamp'],cmap='cool')
# axs[1,1].set_title('Log10 of Abundance [individuals per m$^{{-3}}$]')
axs[2,0].set(xlabel='pH')

axs_6=axs[2,1].scatter(apr_mix['cdom'],apr_mix['Abundance_base10'],10,c = apr_df['timestamp'],cmap='cool')
# axs[1,1].set_title('Log10 of Abundance [individuals per m$^{{-3}}$]')
axs[2,1].set(xlabel='CDOM')

for ax in axs.flat:
    ax.set(ylabel='Log10 of Abundance [individuals per m$^{{-3}}$]')

plt.tight_layout()
plt.savefig("Apriltime.png")

### November 3rd to December 5th

In [ ]:
fig,axs = plt.subplots(3,2, figsize=(15,12))

axs_1=axs[0,0].scatter(nov_mix['temperature'],nov_mix['Abundance_base10'],10,c = nov_df['timestamp'],cmap='cool')
# axs[0,0].set_title('Log10 of Abundance [individuals per m$^{{-3}}$]')
axs[0,0].set(xlabel='Temperature [°C]')

axs_2=axs[0,1].scatter(nov_mix['salinity'],nov_mix['Abundance_base10'],10,c = nov_df['timestamp'],cmap='cool')
# axs[0,1].set_title('Log10 of Abundance [individuals per m$^{{-3}}$]')
axs[0,1].set(xlabel='Salinity [psu]')

axs_3=axs[1,0].scatter(nov_mix['density'],nov_mix['Abundance_base10'],10,c = nov_df['timestamp'],cmap='cool')
# axs[1,0].set_title('Log10 of Abundance [individuals per m$^{{-3}}$]')
axs[1,0].set(xlabel='Density [kg m-1 s-1]')

axs_4=axs[1,1].scatter(nov_mix['chlorophyll_a'],nov_mix['Abundance_base10'],10,c = nov_df['timestamp'],cmap='cool')
# axs[1,1].set_title('Log10 of Abundance [individuals per m$^{{-3}}$]')
axs[1,1].set(xlabel='Chlorophyll [μg L-1]')

axs_5=axs[2,0].scatter(nov_mix['pH'],nov_mix['Abundance_base10'],10,c = nov_df['timestamp'],cmap='cool')
# axs[1,1].set_title('Log10 of Abundance [individuals per m$^{{-3}}$]')
axs[2,0].set(xlabel='pH')

axs_6=axs[2,1].scatter(nov_mix['cdom'],nov_mix['Abundance_base10'],10,c = nov_df['timestamp'],cmap='cool')
# axs[1,1].set_title('Log10 of Abundance [individuals per m$^{{-3}}$]')
axs[2,1].set(xlabel='CDOM')

for ax in axs.flat:
    ax.set(ylabel='Log10 of Abundance [individuals per m$^{{-3}}$]')

plt.tight_layout()
plt.savefig("Novembertime.png")

### Febuary 15th to March 14th

In [ ]:
fig,axs = plt.subplots(3,2, figsize=(15,12))

axs_1=axs[0,0].scatter(feb_mix['temperature'],feb_mix['Abundance_base10'],10,c = feb_df['timestamp'],cmap='cool')
# axs[0,0].set_title('Log10 of Abundance [individuals per m$^{{-3}}$]')
axs[0,0].set(xlabel='Temperature [°C]')

axs_2=axs[0,1].scatter(feb_mix['salinity'],feb_mix['Abundance_base10'],10,c = feb_df['timestamp'],cmap='cool')
# axs[0,1].set_title('Log10 of Abundance [individuals per m$^{{-3}}$]')
axs[0,1].set(xlabel='Salinity [psu]')

axs_3=axs[1,0].scatter(feb_mix['density'],feb_mix['Abundance_base10'],10,c = feb_df['timestamp'],cmap='cool')
# axs[1,0].set_title('Log10 of Abundance [individuals per m$^{{-3}}$]')
axs[1,0].set(xlabel='Density [kg m-1 s-1]')

axs_4=axs[1,1].scatter(feb_mix['chlorophyll_a'],feb_mix['Abundance_base10'],10,c = feb_df['timestamp'],cmap='cool')
# axs[1,1].set_title('Log10 of Abundance [individuals per m$^{{-3}}$]')
axs[1,1].set(xlabel='Chlorophyll [μg L-1]')

axs_5=axs[2,0].scatter(feb_mix['pH'],feb_mix['Abundance_base10'],10,c = feb_df['timestamp'],cmap='cool')
# axs[1,0].set_title('Log10 of Abundance [individuals per m$^{{-3}}$]')
axs[2,0].set(xlabel='pH')

axs_6=axs[2,1].scatter(feb_mix['cdom'],feb_mix['Abundance_base10'],10,c = feb_df['timestamp'],cmap='cool')
# axs[1,1].set_title('Log10 of Abundance [individuals per m$^{{-3}}$]')
axs[2,1].set(xlabel='CDOM')

for ax in axs.flat:
    ax.set(ylabel='Log10 of Abundance [individuals per m$^{{-3}}$]')

plt.tight_layout()
plt.savefig("Febuarytime.png")

## Additional Plots

In [ ]:
# Add some additional packages
import cool_maps.plot as cplt
import cartopy.crs as ccrs
import cmocean.cm as cmo

### Depth Profiles

In [ ]:
# delete a few specified rows at index values 0, 15, 20.
# Note that the index values do not always align to row numbers.
# bm_no_outlier = apr_df.drop(labels=[1065,1142], axis=0)

fig,axs = plt.subplots(3,2, figsize=(15,12))

log10abundance1=axs[0,0].scatter(apr_df['timestamp'],-apr_df['Depth_mean'],10,c = apr_df['Abundance_base10'],cmap=cmo.matter)
axs[0, 0].tick_params(axis='x',rotation=20,labelsize='small')
axs[0, 0].set_title('Spring (Log10 of Abundance [individuals per m$^{{-3}}$])')
plt.colorbar(log10abundance1)

log10biomass1=axs[0,1].scatter(apr_df['timestamp'],-apr_df['Depth_mean'],10,c = apr_df['Biomass_base10'],cmap=cmo.algae)
axs[0, 1].tick_params(axis='x',rotation=20,labelsize='small')
axs[0, 1].set_title('Spring (Log10 of Biomass [grams per m$^{{-3}}$])')
plt.colorbar(log10biomass1)

log10abundance2=axs[1,0].scatter(nov_df['timestamp'],-nov_df['Depth_mean'],10,c = nov_df['Abundance_base10'],cmap=cmo.matter)
axs[1, 0].tick_params(axis='x',rotation=20,labelsize='small')
axs[1, 0].set_title('Fall (Log10 of Abundance [individuals per m$^{{-3}}$])')
plt.colorbar(log10abundance2)

log10biomass2=axs[1,1].scatter(nov_df['timestamp'],-nov_df['Depth_mean'],10,c = nov_df['Biomass_base10'],cmap=cmo.algae)
axs[1, 1].tick_params(axis='x',rotation=20,labelsize='small')
axs[1, 1].set_title('Fall (Log10 of Biomass [grams per m$^{{-3}}$])')
plt.colorbar(log10biomass2)

log10abundance3=axs[2,0].scatter(feb_df['timestamp'],-feb_df['Depth_mean'],10,c = feb_df['Abundance_base10'],cmap=cmo.matter)
axs[2, 0].tick_params(axis='x',rotation=20,labelsize='small')
axs[2, 0].set_title('Winter (Log10 of Abundance [individuals per m$^{{-3}}$])')
plt.colorbar(log10abundance3)

log10biomass3=axs[2,1].scatter(feb_df['timestamp'],-feb_df['Depth_mean'],10,c = feb_df['Biomass_base10'],cmap=cmo.algae)
axs[2, 1].tick_params(axis='x',rotation=20,labelsize='small')
axs[2, 1].set_title('Winter (Log10 of Biomass [grams per m$^{{-3}}$])')
plt.colorbar(log10biomass3)

for ax in axs.flat:
    ax.set(ylabel='Mean Depth [m]')
    
plt.tight_layout()
plt.savefig("scatter.png")

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(15, 10))

temp_2D = axs[0, 0].scatter(DE_420['timestamp'],-DE_420['depth'],10,c=DE_420['temperature'], marker="o", edgecolors='none',cmap = cmo.thermal)
axs[0, 0].tick_params(axis='x',rotation=45,labelsize='small')
axs[0, 0].set_title('Temperature [°C]')
plt.colorbar(temp_2D)

sal_2D = axs[0, 1].scatter(DE_420['timestamp'],-DE_420['depth'],10,c=DE_420['salinity'], marker="o", edgecolors='none',cmap = cmo.haline)
axs[0, 1].tick_params(axis='x',rotation=45,labelsize='small')
axs[0, 1].set_title('Salinity [psu]')
plt.colorbar(sal_2D)

dense_2D = axs[1, 0].scatter(DE_420['timestamp'],-DE_420['depth'],10,c=DE_420['density'], marker="o", edgecolors='none',cmap = cmo.dense)
axs[1, 0].tick_params(axis='x',rotation=45,labelsize='small')
axs[1, 0].set_title('Density [kg m-1 s-1]')
plt.colorbar(dense_2D)

chlor_2D = axs[1, 1].scatter(DE_420['timestamp'],-DE_420['depth'],10,c=DE_420['chlorophyll_a'], marker="o", edgecolors='none',cmap = cmo.algae)
axs[1, 1].tick_params(axis='x',rotation=45,labelsize='small')
axs[1, 1].set_title('Chlorophyll [μg L-1]')
plt.colorbar(chlor_2D)

for ax in axs.flat:
    ax.set(ylabel='Depth [m]')
    
plt.suptitle('April 20th to May 12th')
plt.tight_layout()
plt.savefig("errdap.png")

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(15, 10))

temp_2D = axs[0, 0].scatter(DE_1103['timestamp'],-DE_1103['depth'],10,c=DE_1103['temperature'], marker="o", edgecolors='none',cmap = cmo.thermal)
axs[0, 0].tick_params(axis='x',rotation=45,labelsize='small')
axs[0, 0].set_title('Temperature [°C]')
plt.colorbar(temp_2D)

sal_2D = axs[0, 1].scatter(DE_1103['timestamp'],-DE_1103['depth'],10,c=DE_1103['salinity'], marker="o", edgecolors='none',cmap = cmo.haline)
axs[0, 1].tick_params(axis='x',rotation=45,labelsize='small')
axs[0, 1].set_title('Salinity [psu]')
plt.colorbar(sal_2D)

dense_2D = axs[1, 0].scatter(DE_1103['timestamp'],-DE_1103['depth'],10,c=DE_1103['density'], marker="o", edgecolors='none',cmap = cmo.dense)
axs[1, 0].tick_params(axis='x',rotation=45,labelsize='small')
axs[1, 0].set_title('Density [kg m-1 s-1]')
plt.colorbar(dense_2D)

chlor_2D = axs[1, 1].scatter(DE_1103['timestamp'],-DE_1103['depth'],10,c=DE_1103['chlorophyll_a'], marker="o", edgecolors='none',cmap = cmo.algae)
axs[1, 1].tick_params(axis='x',rotation=45,labelsize='small')
axs[1, 1].set_title('Chlorophyll [μg L-1]')
plt.colorbar(chlor_2D)

for ax in axs.flat:
    ax.set(ylabel='Depth [m]')

plt.suptitle('November 3rd to December 5th')
plt.tight_layout()
plt.savefig("errdap.png")

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(15, 10))

temp_2D = axs[0, 0].scatter(DE_215['timestamp'],-DE_215['depth'],10,c=DE_215['temperature'], marker="o", edgecolors='none',cmap = cmo.thermal)
axs[0, 0].tick_params(axis='x',rotation=45,labelsize='small')
axs[0, 0].set_title('Temperature [°C]')
plt.colorbar(temp_2D)

sal_2D = axs[0, 1].scatter(DE_215['timestamp'],-DE_215['depth'],10,c=DE_215['salinity'], marker="o", edgecolors='none',cmap = cmo.haline)
axs[0, 1].tick_params(axis='x',rotation=45,labelsize='small')
axs[0, 1].set_title('Salinity [psu]')
plt.colorbar(sal_2D)

dense_2D = axs[1, 0].scatter(DE_215['timestamp'],-DE_215['depth'],10,c=DE_215['density'], marker="o", edgecolors='none',cmap = cmo.dense)
axs[1, 0].tick_params(axis='x',rotation=45,labelsize='small')
axs[1, 0].set_title('Density [kg m-1 s-1]')
plt.colorbar(dense_2D)

chlor_2D = axs[1, 1].scatter(DE_215['timestamp'],-DE_215['depth'],10,c=DE_215['chlorophyll_a'], marker="o", edgecolors='none',cmap = cmo.algae)
axs[1, 1].tick_params(axis='x',rotation=45,labelsize='small')
axs[1, 1].set_title('Chlorophyll [μg L-1]')
plt.colorbar(chlor_2D)

for ax in axs.flat:
    ax.set(ylabel='Depth [m]')

plt.suptitle('Febuary 15th to March 14th')
plt.tight_layout()
plt.savefig("errdap.png")

### Cartopy Maps

In [ ]:
extent     = [-75, -72.5, 38.4, 40.5]
markersize = 5;

fig,ax  = cplt.create(extent, proj=ccrs.Mercator(), bathymetry=True, isobaths=(-1000, -500, -100, -50, -20, -10), figsize=(10,6), oceancolor='w')
ax_posi = plt.scatter(apr_df['Lon_M'],apr_df['Lat_M'],apr_df['Depth_mean'],c=apr_df['timestamp'], marker="o", edgecolors='none',cmap='cool', transform=ccrs.PlateCarree())

# Define your mappable for colorbar creation
sm = plt.cm.ScalarMappable(cmap='cool', 
                           norm=plt.Normalize(vmin=apr_df['timestamp'].min().value,
                                              vmax=apr_df['timestamp'].max().value));
# Add the colorbar
sm._A = [];
cbar = plt.colorbar(sm, ax=ax);

# Change the numeric ticks into ones that match. Time format options detailed here: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior
cbar.ax.set_yticklabels(pd.to_datetime(cbar.get_ticks()).strftime(date_format='%b %d'));
plt.savefig("map1.png")

In [ ]:
extent     = [-75, -72.5, 38.4, 40.5]
markersize = 5;

fig,ax  = cplt.create(extent, proj=ccrs.Mercator(), bathymetry=True, isobaths=(-1000, -500, -100, -50, -20, -10), figsize=(10,6), oceancolor='w')
ax_posi = plt.scatter(nov_df['Lon_M'],nov_df['Lat_M'],nov_df['Depth_mean'],c=nov_df['timestamp'], marker="o", edgecolors='none',cmap='cool', transform=ccrs.PlateCarree())

# Define your mappable for colorbar creation
sm = plt.cm.ScalarMappable(cmap='cool', 
                           norm=plt.Normalize(vmin=apr_df['timestamp'].min().value,
                                              vmax=apr_df['timestamp'].max().value));
# Add the colorbar
sm._A = [];
cbar = plt.colorbar(sm, ax=ax);

# Change the numeric ticks into ones that match. Time format options detailed here: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior
cbar.ax.set_yticklabels(pd.to_datetime(cbar.get_ticks()).strftime(date_format='%b %d'));
plt.savefig("map2.png")

In [ ]:
extent     = [-75, -72.5, 38.4, 40.5]
markersize = 5;

fig,ax  = cplt.create(extent, proj=ccrs.Mercator(), bathymetry=True, isobaths=(-1000, -500, -100, -50, -20, -10), figsize=(10,6), oceancolor='w')
ax_posi = plt.scatter(feb_df['Lon_M'],feb_df['Lat_M'],feb_df['Depth_mean'],c=feb_df['timestamp'], marker="o", edgecolors='none',cmap='cool', transform=ccrs.PlateCarree())

# Define your mappable for colorbar creation
sm = plt.cm.ScalarMappable(cmap='cool', 
                           norm=plt.Normalize(vmin=apr_df['timestamp'].min().value,
                                              vmax=apr_df['timestamp'].max().value));
# Add the colorbar
sm._A = [];
cbar = plt.colorbar(sm, ax=ax);

# Change the numeric ticks into ones that match. Time format options detailed here: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior
cbar.ax.set_yticklabels(pd.to_datetime(cbar.get_ticks()).strftime(date_format='%b %d'));
plt.savefig("map3.png")

### Raw Depth vs. MLD

In [ ]:
fig,axs = plt.subplots(3, figsize=(10,12))

axs_1=axs[0].scatter(apr_mix_mld['mld'],apr_mix_mld['Biomass_base10'],5,c='red')
axs[0].set(xlabel='Mixed Layer Depth [4/20 - 5/12]')
axs[0].set(ylabel='Log10 of Biomass [grams per m$^{{-3}}$]')
axs_2=axs[1].scatter(nov_mix_mld['mld'],nov_mix_mld['Biomass_base10'],5,c='blue')
axs[1].set(xlabel='Mixed Layer Depth [11/3 - 12/5]')
axs[1].set(ylabel='Log10 of Biomass [grams per m$^{{-3}}$]')
axs_3=axs[2].scatter(feb_mix_mld['mld'],feb_mix_mld['Biomass_base10'],5,c='green')
axs[2].set(xlabel='Mixed Layer Depth [2/15 - 3/14]')
axs[2].set(ylabel='Log10 of Biomass [grams per m$^{{-3}}$]')
plt.tight_layout()
plt.savefig("mld.png")